# 01 — EDA avançado: missingness, outliers e validações

Objetivo: deixar sua EDA mais robusta e reprodutível.

Tempo: ~25–30 min

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

In [ ]:
import pandas as pd

df = pd.read_csv(SAMPLE_DIR / "sales.csv")
df["date"] = pd.to_datetime(df["date"])
df.head()

## 1) Missingness (nulos)

In [ ]:
missing = df.isna().sum().sort_values(ascending=False)
missing

## 2) Outliers (IQR)

Regra prática: identificar possíveis pontos fora do padrão para investigar, não para excluir automaticamente.

In [ ]:
import numpy as np

q1 = df["revenue"].quantile(0.25)
q3 = df["revenue"].quantile(0.75)
iqr = q3 - q1
low = q1 - 1.5 * iqr
high = q3 + 1.5 * iqr

outliers = df[(df["revenue"] < low) | (df["revenue"] > high)]
low, high, outliers.shape[0]

## 3) Validações mínimas (qualidade)

Você vai usar isso em todo projeto.

In [ ]:
report = {
    "rows": int(len(df)),
    "duplicated_order_id": int(df.duplicated("order_id").sum()),
    "revenue_le_zero": int((df["revenue"] <= 0).sum()),
    "unit_price_le_zero": int((df["unit_price"] <= 0).sum()),
}

report

## 4) Exportar um relatório simples em JSON

In [ ]:
import json

out_dir = ROOT / "data" / "output"
out_dir.mkdir(parents=True, exist_ok=True)

(out_dir / "eda_report.json").write_text(json.dumps(report, ensure_ascii=False, indent=2), encoding="utf-8")
out_dir / "eda_report.json"

## Exercícios

1- Faça outliers por `category` (IQR por grupo).
2- Crie uma função `validate_df(df)` que retorna um report.
3- Aponte 2 hipóteses de negócio com base nos outliers/segmentações.

In [ ]:
# Escreva suas respostas aqui
